# $\chi^2$ statistics for constant $\beta$ 

In [1]:
import sys

sys.path.append('../')
import rings2cosmo
import pandas as pd
import numpy as np



In [2]:
data = pd.read_csv('../data/sample80.csv')
z_L = data['zl'].values
z_S = data['zs'].values
theta_E = (data['theta_E'].values * rings2cosmo.u.arcsec).to(rings2cosmo.u.rad).value
theta_ap = (data['theta_ap'].values * rings2cosmo.u.arcsec).to(rings2cosmo.u.rad).value
seeing_atm = (data['sigma_atm'].values * rings2cosmo.u.arcsec).to(rings2cosmo.u.rad).value
velDisp = data['sigma_ap'].values
velDispErr = data['d_sigma_ap'].values

First the $\chi^2$ is given by

$$
\chi^2 = c - 2ln(\mathfrak{L}(\mathbf{X_{fitted}})) = \sum\left(\frac{(\sigma_{SDSS} - \sigma_*)^2}{\varepsilon^2_{SDSS}}\right)
$$

but I set $c$ so it is zero.

For constant $\beta$ the parameters $\mathbf{X}$ are $\alpha$, $\delta$, $\beta_0$ and $\gamma$ 


In [3]:
def chi_2(theta, z_S, z_L, velDisp, velDispErr, theta_E, seeing_atm, theta_ap):
    
    alpha, beta, delta, gamma = theta
    model = rings2cosmo.vel(z_S, z_L, theta_E, seeing_atm,
                theta_ap, alpha, beta, gamma, delta)
    return  np.sum((velDisp - model) ** 2 / (velDispErr ** 2))

In [4]:
#The values fitted are
alpha = 2.0
delta = 2.4
beta0 = 0.18 
gamma = 0.998

X = (alpha, beta0, delta, gamma)

In [5]:
chi2 = chi_2(X, z_S, z_L, velDisp, velDispErr, theta_E, seeing_atm, theta_ap)

In [6]:
chi2

<Quantity 186.19384969>


$$
\chi^2/(n - df)
$$
where n is the sample size and df is the degree of freedom 
$$
n = 80
$$
 and for this case, if we consider priors on $\alpha$, $\delta$ and $\beta_0$, then the only free paramer is $\gamma_{PPN}$, and therefore 
 $$
 df = 1
 $$

In [7]:
n = 80
df = 1.0
chi_eff= chi2/(n - df)
chi_eff

<Quantity 2.35688417>

In [8]:
#for case where one needs to compare the result cross models
pars_df = pd.DataFrame({"Parameter": [r"$\alpha$", r"$\delta$", r"$\beta_0$", "a", r"$\gamma_{PPN}$", r"$\chi^2$", r"$\chi^2_{eff}$"],
            "cte_cao": [alpha, delta, beta0, 0, gamma, chi2, chi_eff]})
pars_df

,Parameter,cte_cao
0,$\alpha$,2.0
1,$\delta$,2.4
2,$\beta_0$,0.18
3,a,0
4,$\gamma_{PPN}$,0.998
5,$\chi^2$,186.1938496949082
6,$\chi^2_{eff}$,2.3568841733532686


In [9]:
# pars_df.to_csv("cte_cao.csv", index = False)

# BIC

The bayesian information criterium is given by
$$ BIC = df*\ln(N) + \chi^2 $$

In [10]:
((np.log(80.0))*df) + chi2

<Quantity 190.57587633>